In [7]:
import math
from scripts_stock.cfg.out_file_name import OutFileName
from scripts_stock.cfg.set_dir import ProjectDir
from scripts_stock.cfg.stock_list import *
from scripts_stock.utils.analysis.stock_stat_index import df_to_stock_df, stock_kdj
import os
from scripts_stock.utils.common import CommonScript
from scripts_stock.utils.string_process import StringProcess

def get_data_sql_str(stock_index):
    return f"""SELECT *  FROM prd_t_fuquan_dfcf  where stock_index= {stock_index}"""

def get_all_stock_index_sql_str():
    return f"""SELECT distinct stock_index  FROM prd_t_fuquan_dfcf """

def get_fuquan_stock_index_sql_str():
    return """
                SELECT distinct stock_index 
                FROM prd_t_fuquan_dfcf  
            """

def get_stock_indicator(stock_df_in):
    stock_df = df_to_stock_df(stock_df_in[["open","close","high","low","stock_index","stock_date"]])
    stock_kdj_ind, _ = stock_kdj(stock_df)
    print(stock_kdj_ind)
    #stock_kdj_ind["stock_index"] = stock_df_in["stock_index"].values[0]
    return stock_kdj_ind

In [8]:
conn = CommonScript.connect_to_db("test.db")
cursor = conn.cursor()
stock_list = pd.read_sql_query(get_all_stock_index_sql_str(),conn)
print(stock_list)

     stock_index
0         600011
1            895
2         600010
3         600660
4         601728
..           ...
246          877
247       600061
248          568
249       600584
250         2460

[251 rows x 1 columns]


In [7]:
conn = CommonScript.connect_to_db("test.db")
cursor = conn.cursor()
stock_index = "601398"
input_sql_str = get_data_sql_str(stock_index)
df1 = pd.read_sql_query(input_sql_str, conn)
df1["stock_index"] = StringProcess.int_to_stock_index(df1["stock_index"].values[0])
df1['stock_date'] = df1["date"]
stock_kdj_ind = get_stock_indicator(df1)
stock_kdj_ind_tail = stock_kdj_ind.tail(5)
stock_kdj_ind_tail

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


           date  open  close  high   low stock_index  date_time  stock_date  \
0    2012-05-10  1.05   1.05  1.07  1.03      601398 2012-05-10  2012-05-10   
1    2012-05-11  1.03   1.01  1.06  1.00      601398 2012-05-11  2012-05-11   
2    2012-05-14  1.02   1.02  1.05  1.00      601398 2012-05-14  2012-05-14   
3    2012-05-15  1.01   0.98  1.02  0.97      601398 2012-05-15  2012-05-15   
4    2012-05-16  0.97   0.95  1.01  0.93      601398 2012-05-16  2012-05-16   
...         ...   ...    ...   ...   ...         ...        ...         ...   
2995 2024-09-02  5.90   5.94  6.00  5.88      601398 2024-09-02  2024-09-02   
2996 2024-09-03  5.93   5.76  5.95  5.68      601398 2024-09-03  2024-09-03   
2997 2024-09-04  5.72   5.71  5.78  5.66      601398 2024-09-04  2024-09-04   
2998 2024-09-05  5.70   5.63  5.73  5.60      601398 2024-09-05  2024-09-05   
2999 2024-09-06  5.63   5.64  5.73  5.62      601398 2024-09-06  2024-09-06   

         macdh         cci  ...     rsi_24       kd

,date,open,close,high,low,stock_index,date_time,stock_date,macdh,cci,...,rsi_24,kdjk,kdjd,kdjj,boll_ub,boll_lb,macd,macds,wr_6,wr_10
2995,2024-09-02,5.90,5.94,6.00,5.88,601398,2024-09-02,2024-09-02,-0.046923,-127.395747,...,49.911391,34.268365,58.988632,-15.172168,6.637044,5.666956,0.092792,0.139714,91.304348,91.304348
2996,2024-09-03,5.93,5.76,5.95,5.68,601398,2024-09-03,2024-09-03,-0.071381,-162.197253,...,45.111538,25.841832,47.939699,-18.353902,6.641443,5.658557,0.050488,0.121869,91.011236,91.011236
2997,2024-09-04,5.72,5.71,5.78,5.66,601398,2024-09-04,2024-09-04,-0.087272,-159.669170,...,43.888171,19.059390,38.312929,-19.447689,6.653637,5.635363,0.012779,0.100051,94.505495,94.505495
2998,2024-09-05,5.70,5.63,5.73,5.60,601398,2024-09-05,2024-09-05,-0.098674,-141.333333,...,41.987142,13.737188,30.121015,-19.030468,6.678832,5.586168,-0.023292,0.075382,96.629213,96.907216
2999,2024-09-06,5.63,5.64,5.73,5.62,601398,2024-09-06,2024-09-06,-0.100697,-110.321925,...,42.313063,10.532695,23.591575,-15.585064,6.700919,5.536081,-0.050489,0.050208,93.750000,95.876289


In [10]:
dir1 = os.path.join(ProjectDir.analysis_data_dir,"stock_kdj_last.csv")
stock_kdj_ind_tail.round(3).to_csv(dir1,index=0)

In [13]:
df1.groupby(lambda x:math.floor(x/30)).last()

,date,open,close,high,low,stock_index,update_time,stock_date
0,2012-06-21,0.79,0.76,0.80,0.75,601398,2024-09-09 18:21:56.527726,2012-06-21
1,2012-08-03,0.63,0.64,0.66,0.63,601398,2024-09-09 18:21:56.527726,2012-08-03
2,2012-09-14,0.61,0.63,0.64,0.60,601398,2024-09-09 18:21:56.527726,2012-09-14
3,2012-11-02,0.72,0.76,0.78,0.71,601398,2024-09-09 18:21:56.527726,2012-11-02
4,2012-12-14,0.80,0.95,0.98,0.79,601398,2024-09-09 18:21:56.527726,2012-12-14
...,...,...,...,...,...,...,...,...
95,2024-03-14,4.85,4.88,4.92,4.83,601398,2024-09-09 18:21:56.527726,2024-03-14
96,2024-04-29,5.08,5.07,5.13,5.03,601398,2024-09-09 18:21:56.527726,2024-04-29
97,2024-06-14,5.12,5.19,5.21,5.11,601398,2024-09-09 18:21:56.527726,2024-06-14
98,2024-07-26,6.05,5.93,6.07,5.86,601398,2024-09-09 18:21:56.527726,2024-07-26


In [10]:
conn = CommonScript.connect_to_db("test.db")
cursor = conn.cursor()
stock_index = "601398"
input_sql_str = get_data_sql_str(stock_index)
df1 = pd.read_sql_query(input_sql_str, conn)
df1.tail(10)

,date,open,close,high,low,stock_index,update_time
2990,2024-09-05,5.70,5.63,5.73,5.60,601398,2024-09-22 00:55:19.943886
2991,2024-09-06,5.63,5.64,5.73,5.62,601398,2024-09-22 00:55:19.943886
2992,2024-09-09,5.63,5.61,5.65,5.51,601398,2024-09-22 00:55:19.943886
2993,2024-09-10,5.61,5.67,5.72,5.60,601398,2024-09-22 00:55:19.943886
2994,2024-09-11,5.66,5.54,5.67,5.50,601398,2024-09-22 00:55:19.943886
2995,2024-09-12,5.51,5.54,5.56,5.45,601398,2024-09-22 00:55:19.943886
2996,2024-09-13,5.54,5.56,5.58,5.52,601398,2024-09-22 00:55:19.943886
2997,2024-09-18,5.57,5.68,5.68,5.57,601398,2024-09-22 00:55:19.943886
2998,2024-09-19,5.69,5.65,5.71,5.58,601398,2024-09-22 00:55:19.943886
2999,2024-09-20,5.66,5.67,5.69,5.62,601398,2024-09-22 00:55:19.943886


In [11]:
df1['diff'] = df1['close'].shift(-1) - df1['close']
df1

,date,open,close,high,low,stock_index,update_time,diff
0,2012-05-22,0.88,0.88,0.89,0.85,601398,2024-09-22 00:55:19.943886,-0.05
1,2012-05-23,0.87,0.83,0.88,0.82,601398,2024-09-22 00:55:19.943886,0.01
2,2012-05-24,0.83,0.84,0.85,0.82,601398,2024-09-22 00:55:19.943886,-0.01
3,2012-05-25,0.83,0.83,0.85,0.81,601398,2024-09-22 00:55:19.943886,0.02
4,2012-05-28,0.83,0.85,0.86,0.81,601398,2024-09-22 00:55:19.943886,0.02
...,...,...,...,...,...,...,...,...
2995,2024-09-12,5.51,5.54,5.56,5.45,601398,2024-09-22 00:55:19.943886,0.02
2996,2024-09-13,5.54,5.56,5.58,5.52,601398,2024-09-22 00:55:19.943886,0.12
2997,2024-09-18,5.57,5.68,5.68,5.57,601398,2024-09-22 00:55:19.943886,-0.03
2998,2024-09-19,5.69,5.65,5.71,5.58,601398,2024-09-22 00:55:19.943886,0.02
